In [2]:
# Importer les bibliothèques nécessaires
import numpy as np
import random

# Définir les paramètres du code BIKE-1
n = 20326 # Longueur du code
r = n//2 # Longueur des polynômes h0 et h1
w = 71 # Poids de Hamming des polynômes h0 et h1
t = 134 # Seuil de bit-flipping
e = 0.2 # Taux d'erreur

# Générer aléatoirement les polynômes h0 et h1 de poids w
def gen_h():
    h = np.zeros(r, dtype=int) # Créer un vecteur nul de taille r
    idx = random.sample(range(r), w) # Choisir aléatoirement w indices
    h[idx] = 1 # Mettre ces indices à 1
    return h

# Calculer le produit de deux polynômes modulo x^r - 1
def poly_mul(a, b):
    c = np.convolve(a, b) # Calculer la convolution linéaire
    c = np.mod(c, 2) # Réduire les coefficients modulo 2
    c = np.append(c[r:], c[:r]) # Effectuer la réduction modulo x^r - 1
    return c

# Calculer le syndrome d'un polynôme binaire c
def syndrome(c, h0, h1):
    c0 = c[:r] # Extraire la première moitié de c
    c1 = c[r:] # Extraire la deuxième moitié de c
    s = poly_mul(h0, c0) + poly_mul(h1, c1) # Calculer le syndrome
    s = np.mod(s, 2) # Réduire les coefficients modulo 2
    return s

# Appliquer l'algorithme de bit-flipping pour décoder c
def bit_flipping(c, h0, h1):
    s = syndrome(c, h0, h1) # Calculer le syndrome initial
    e = np.zeros(n, dtype=int) # Initialiser le vecteur d'erreur
    while np.count_nonzero(s) > t: # Tant que le poids de Hamming du syndrome est supérieur au seuil
        for i in range(n): # Pour chaque bit de c
            ci = c[i] # Sauvegarder le bit courant
            c[i] = 1 - c[i] # Inverser le bit
            si = syndrome(c, h0, h1) # Calculer le nouveau syndrome
            if np.count_nonzero(si) < np.count_nonzero(s): # Si le poids de Hamming du syndrome a diminué
                s = si # Mettre à jour le syndrome
                e[i] = 1 - e[i] # Mettre à jour le vecteur d'erreur
            else: # Sinon
                c[i] = ci # Rétablir le bit
    return e # Retourner le vecteur d'erreur

# Générer les polynômes h0 et h1
h0 = gen_h()
h1 = gen_h()

# Générer un chiffré c aléatoire avec un taux d'erreur e
c = np.random.binomial(1, e, n)

# Décoder c avec l'algorithme de bit-flipping
e = bit_flipping(c, h0, h1)

# Afficher le résultat
print("Le vecteur d'erreur est :")
print(e)
